This notebook downloads MACA V2 data for shey_gdf (Sheyenne National Grassland)

In [7]:
import os
import pathlib
import geopandas as gpd

import pandas as pd
import rioxarray as rxr

# Import packages
import numpy as np
import netCDF4
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import seaborn as sns

In [8]:
%store -r shey_gdf

In [9]:
model_name = 'BNU-ESM'
variable_name = 'pr'
var_long_name = 'precipitation'
dir_path = 'http://thredds.northwestknowledge.net:8080/thredds/dodsC/'

In [10]:
# build a directory for the macav2 data

shey_path = os.path.join(
    pathlib.Path.home(),
    'earth-analytics',
    'data',
    'habitat_suitability2'
)
os.makedirs(shey_path, exist_ok=True)
# shey_gdf.to_file('habitat_suitability')
# shey_gdf = gpd.read_file(shey_path)

# Define the file path for the GeoDataFrame
shey_file = os.path.join(shey_path, 'shey_data.shp')

# Save the GeoDataFrame to the file
shey_gdf.to_file(shey_file)

# Read the GeoDataFrame back from the file
shey_gdf = gpd.read_file(shey_file)

In [11]:
def convert_longitude(longitude):
        """Convert longitude from 0-360 to -180-180"""
        return (longitude - 360) if longitude > 180 else longitude

maca_da_list = []
for site_name, site_gdf in {'shey': shey_gdf}.items():
        for variable in ["pr"]:
                for start_year in [2096]: # I'm confused about how this feeds into the fuzzy logic model.
                        end_year = start_year + 3 # Did I use the wrong start and end year here?
                        maca_url = (
                                "http://thredds.northwestknowledge.net:8080/thredds"
                                f"/dodsC/MACAV2/BNU-ESM/macav2metdata_{variable}_BNU-ESM_"
                                f"r1i1p1_rcp85_{start_year}_{end_year}_CONUS_monthly.nc"
                        )
                        maca_da = xr.open_dataset(maca_url).squeeze().precipitation
                        bounds = site_gdf.to_crs(maca_da.rio.crs).total_bounds
                        maca_da = maca_da.assign_coords(
                                lon=("lon", [convert_longitude(l) for l in maca_da.lon.values])
                        )
                        maca_da = maca_da.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
                        maca_da.rio.clip_box(*bounds)
                        maca_da_list.append(dict(
                                site_name=site_name,
                                variable=variable,
                                start_year=start_year,
                                da=maca_da))
pd.DataFrame(maca_da_list)

,site_name,variable,start_year,da
0,shey,pr,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [12]:
maca_df = pd.DataFrame(maca_da_list)
maca_df

,site_name,variable,start_year,da
0,shey,pr,2096,[[[<xarray.DataArray 'precipitation' ()> Size:...


In [13]:
%store maca_df

Stored 'maca_df' (DataFrame)


In [14]:
maca_da.rio.clip_box(*shey_gdf.to_crs(maca_da.rio.crs).total_bounds)
maca_da.rio.clip_box(*bounds)

<xarray.DataArray 'precipitation' (time: 48, lat: 12, lon: 14)> Size: 32kB
[8064 values with dtype=float32]
Coordinates:
  * lat      (lat) float64 96B 46.1 46.15 46.19 46.23 ... 46.48 46.52 46.56
  * time     (time) object 384B 2096-01-15 00:00:00 ... 2099-12-15 00:00:00
  * lon      (lon) float64 112B -97.48 -97.44 -97.4 ... -97.02 -96.98 -96.94
    crs      int64 8B 0
Attributes:
    long_name:      Monthly Precipitation Amount
    units:          mm
    standard_name:  precipitation
    cell_methods:   time: sum(interval: 24 hours): sum over days
    comments:       Total monthly precipitation at surface: includes both liq...
    _ChunkSizes:    [  8 115 276]